In [1]:
import sys
sys.path.append("../../")

from objects.node import Node
from objects.relationship import Relationship
from objects.data_model import DataModel
from ingestion.generate_ingest import IngestionGenerator

In [2]:
uri = "bolt://localhost:7687"
username = "neo4j"
password = "password"
database = "neo4j"

In [3]:
good_nodes = [
            Node(
                label="Person",
                properties=["name", "age"],
                unique_constraints=["name"],
                ),
            Node(
                label="Address", 
                properties=["street", "city"], 
                unique_constraints=[]
            ),
            Node(
                label="Pet", 
                properties=["name", "kind"], 
                unique_constraints=[]
            ),
            Node(
                label="Toy", 
                properties=["name", "kind"], 
                unique_constraints=["name"]
            )
        ]

good_relationships = [
            Relationship(
                type="HAS_ADDRESS",
                properties=[],
                unique_constraints=[],
                source="Person",
                target="Address",
            ),
            Relationship(
                type="KNOWS",
                unique_constraints=[],
                properties=[],
                source="Person",
                target="Person",
            ),
            Relationship(
                type="HAS_PET",
                unique_constraints=[],
                properties=[],
                source="Person",
                target="Pet",
            ),
            Relationship(
                type="PLAYS_WITH",
                unique_constraints=[],
                properties=[],
                source="Pet",
                target="Toy",
            )
        ]

data_model = DataModel(nodes=good_nodes, relationships=good_relationships)

In [4]:
gen = IngestionGenerator(data_model=data_model.dict, 
                         username=username, 
                         password=password, 
                         uri=uri, 
                         database=database, 
                         csv_file_path=None)

In [5]:
print(gen.generate_pyingest_yaml(write_file=False))

server_uri: bolt://localhost:7687
admin_user: neo4j
admin_pass: password
basepath: file:./

pre_ingest:
  - CREATE CONSTRAINT person_name IF NOT EXISTS FOR (n:Person) REQUIRE n.name IS UNIQUE;
  - CREATE CONSTRAINT address_name IF NOT EXISTS FOR (n:Address) REQUIRE n.name IS UNIQUE;
  - CREATE CONSTRAINT pet_name IF NOT EXISTS FOR (n:Pet) REQUIRE n.name IS UNIQUE;
  - CREATE CONSTRAINT toy_name IF NOT EXISTS FOR (n:Toy) REQUIRE n.name IS UNIQUE;
files:
- chunk_size: 100
  cql: |-
    WITH $dict.rows AS rows
    UNWIND rows AS row
    MERGE (n:Person {name: row.name})
    SET n.age = row.age
  url: $BASE/data/None
- chunk_size: 100
  cql: |-
    WITH $dict.rows AS rows
    UNWIND rows AS row
    MERGE (n:Address {name: row.name})
    SET n.street = row.street, n.city = row.city
  url: $BASE/data/None
- chunk_size: 100
  cql: |-
    WITH $dict.rows AS rows
    UNWIND rows AS row
    MERGE (n:Pet {name: row.name})
    SET n.kind = row.kind, n.name = row.name
  url: $BASE/data/None
- chunk

In [6]:
print(gen.generate_constraints_cypher())

CREATE CONSTRAINT person_name IF NOT EXISTS FOR (n:Person) REQUIRE n.name IS UNIQUE;
CREATE CONSTRAINT address_name IF NOT EXISTS FOR (n:Address) REQUIRE n.name IS UNIQUE;
CREATE CONSTRAINT pet_name IF NOT EXISTS FOR (n:Pet) REQUIRE n.name IS UNIQUE;
CREATE CONSTRAINT toy_name IF NOT EXISTS FOR (n:Toy) REQUIRE n.name IS UNIQUE;



In [7]:
print(gen.generate_load_csv())

CREATE CONSTRAINT person_name IF NOT EXISTS FOR (n:Person) REQUIRE n.name IS UNIQUE;
CREATE CONSTRAINT address_name IF NOT EXISTS FOR (n:Address) REQUIRE n.name IS UNIQUE;
CREATE CONSTRAINT pet_name IF NOT EXISTS FOR (n:Pet) REQUIRE n.name IS UNIQUE;
CREATE CONSTRAINT toy_name IF NOT EXISTS FOR (n:Toy) REQUIRE n.name IS UNIQUE;
LOAD CSV WITH HEADERS FROM 'file:///file_name' as row
CALL {
	WITH row
	MERGE (n:Person {name: row.name})
SET n.age = row.age} IN TRANSACTIONS OF 10000 ROWS;
LOAD CSV WITH HEADERS FROM 'file:///file_name' as row
CALL {
	WITH row
	MERGE (n:Address {name: row.name})
SET n.street = row.street, n.city = row.city} IN TRANSACTIONS OF 10000 ROWS;
LOAD CSV WITH HEADERS FROM 'file:///file_name' as row
CALL {
	WITH row
	MERGE (n:Pet {name: row.name})
SET n.kind = row.kind, n.name = row.name} IN TRANSACTIONS OF 10000 ROWS;
LOAD CSV WITH HEADERS FROM 'file:///file_name' as row
CALL {
	WITH row
	MERGE (n:Toy {name: row.name})
SET n.kind = row.kind} IN TRANSACTIONS OF 10000 R